# 참고
- https://pytorch.org/docs/0.4.0/nn.html
- https://pytorch.org/tutorials/
- https://ratsgo.github.io/machine%20learning/2017/10/12/terms/
- https://github.com/DSKSD/Pytorch_Fast_Campus_2018

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import numpy as np
from collections import OrderedDict

torch.manual_seed(1)

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# What is a PyTorch?
- Tensor와 Optimizer, Neural Net등 GPU연산에 최적화된 모듈을 이용하여 빠르게 딥러닝 모델을 구현할 수 있는 프레임워크
Facebook이 밀고 있던 lua기반의 torch를 python버전으로 포팅함.

# > Pytorch Basic
- Tensor
- autograd
- nn
- optim

## 3. nn Module
- nn.Module는 모든 neural network의 기본 class임.
- nn.Module에는 레이어와 출력을 반환하는 메서드 forward가 포함되어있음.
- 미리 포함된 forward함수에 input을 인자로 forward를 계산하고 Parameters의 backward를 계산할 수 있음. 그리고 parameter.grad를 생성함.

### 3.1 nn.Module로 Neural Network생성하기

In [2]:
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__() # 부모클래스 초기화
        self.linear= nn.Linear(2,2) # nn.Linear부분도 다 nn.Module의 자식클래스
        self.sigmoid= nn.Sigmoid() 
        
    def forward(self, inputs):
        outputs= self.linear(inputs)
        outputs= self.sigmoid(outputs)
        
        return outputs

testnn= NN()

print(testnn)
inputs=torch.FloatTensor([1,2])

NN(
  (linear): Linear(in_features=2, out_features=2, bias=True)
  (sigmoid): Sigmoid()
)


### 3.2 생성한 NN에 파라미터 혹은 서브 모듈에 접근
- module class의 parameters메서드는 순차적으로 parameter를 반환하는 generator를 생성

In [3]:
for param in testnn.parameters():
    print(param)

Parameter containing:
tensor([[ 0.3643, -0.3121],
        [-0.1371,  0.3319]], requires_grad=True)
Parameter containing:
tensor([-0.6657,  0.4241], requires_grad=True)


- naemd_parameters는 순차적으로 parameter name과 parameter를 반환하는 generator를 생성


In [4]:
for name, param in testnn.named_parameters():
    print('[',name,']', param)

[ linear.weight ] Parameter containing:
tensor([[ 0.3643, -0.3121],
        [-0.1371,  0.3319]], requires_grad=True)
[ linear.bias ] Parameter containing:
tensor([-0.6657,  0.4241], requires_grad=True)


In [5]:
for child in testnn.children():
    print(child)

Linear(in_features=2, out_features=2, bias=True)
Sigmoid()


### 3.3 forward
- 순차적으로 neural network를 계산함.

In [6]:
inputs= torch.randn(3,2) # nn.Module은 기본적으로 dim=0는 batch size로 봄.
print(inputs)
print("="*50)
print(testnn(inputs))

tensor([[ 0.5636,  1.1431],
        [ 0.8590,  0.7056],
        [-0.3406, -1.2720]])
tensor([[0.3064, 0.6740],
        [0.3606, 0.6319],
        [0.4030, 0.5121]], grad_fn=<SigmoidBackward>)


### 3.4 nn.Sequential()
- 여러 nn.Module을 순차적으로 처리할 수 있는 클래스를 생성.
- 여러 모듈을 간단하게 관리할 수 있음.

In [7]:
model= nn.Sequential(nn.Linear(2,2),
                    nn.Sigmoid())
print(model)
print(model(inputs))

Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): Sigmoid()
)
tensor([[0.6181, 0.2806],
        [0.6382, 0.3031],
        [0.3523, 0.5812]], grad_fn=<SigmoidBackward>)


- OrderedDict로 이름과 함께 관리할 수 있음.

In [8]:
model = nn.Sequential(OrderedDict([
          ('conv1', nn.Conv2d(1,20,5)),
          ('relu1', nn.ReLU()),
          ('conv2', nn.Conv2d(20,64,5)),
          ('relu2', nn.ReLU())
        ])) 
print(model)

Sequential(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (relu1): ReLU()
  (conv2): Conv2d(20, 64, kernel_size=(5, 5), stride=(1, 1))
  (relu2): ReLU()
)
